Hello! In this notebook we will analyse various wine reviews by various sommeliers.This dataset contains over 130,000 different wine reviews, and to avoid memory problems we will analyse the dataset in sets of 5000. The aim is to build a model that can accurately determine the price of the wine from the given data.

In [1]:
# Let us import the necessary libraries

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates
from datetime import datetime
import random

In [2]:
# Importing our dataset, and split our dataset into groups of 5000 records each

# Read the CSV file into a DataFrame
dataset = pd.read_csv('winemag-data_first150k.csv')

# Shuffle the DataFrame
dataset = dataset.sample(frac=1).reset_index(drop=True)

# Split the DataFrame into groups of 5000 records each
group_size = 15000
groups = [dataset[i:i + group_size] for i in range(0, len(dataset), group_size)]



In [3]:
# Now let's start with the first group

data_first = groups[0]
data_first.columns


Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [4]:
# Let's see the number of missing values in each column


null_counts = data_first.isnull().sum()
print(null_counts)


Unnamed: 0        0
country           0
description       0
designation    4494
points            0
price          1376
province          0
region_1       2500
region_2       8980
variety           0
winery            0
dtype: int64


In [5]:
# Let's delete the columns 'designation', 'region_1', and 'region_2', since there are too many missing records. 
# We will also get rid of the description column since we are not using sentiment analysis yet...
columns_to_drop = ['designation', 'region_1', 'region_2', 'description', 'Unnamed: 0']
data_first = data_first.drop(columns_to_drop, axis=1)
null_counts = data_first.isnull().sum()
print(null_counts)

country        0
points         0
price       1376
province       0
variety        0
winery         0
dtype: int64


In [6]:
# Now, let's get rid of any records that are missing values in the 'Price' column

data_first.dropna(inplace=True)
null_counts = data_first.isnull().sum()
print(null_counts)
print("Number of records:", len(data_first))

country     0
points      0
price       0
province    0
variety     0
winery      0
dtype: int64
Number of records: 13624


In [7]:
print(data_first.head(20))


       country  points  price                 province             variety  \
0           US      83   10.0               California     Sauvignon Blanc   
1           US      94   34.0               Washington               Syrah   
2    Argentina      87   17.0         Mendoza Province  Cabernet Sauvignon   
3        Spain      92   41.0           Northern Spain         Tempranillo   
4        Chile      85   13.0         Aconcagua Valley  Cabernet Sauvignon   
6           US      86   16.0               California          Chardonnay   
7        Spain      82   12.0            Central Spain                Rosé   
8           US      86   10.0               California     Sauvignon Blanc   
9     Portugal      84   12.0                     Port                Port   
10          US      87    8.0               California     Sauvignon Blanc   
11          US      91   38.0               California          Chardonnay   
12          US      85   20.0               Washington        Pe

In [8]:
# Let us see the number of unique entries in each column, and only keep the unique entries which appear at least 

country_counts = data_first['country'].value_counts()
print(country_counts)


country
US                6135
Italy             1842
France            1444
Spain              873
Chile              599
Argentina          555
Australia          493
Portugal           437
New Zealand        298
Austria            234
Germany            221
South Africa       203
Greece              94
Israel              59
Hungary             26
Canada              25
Romania             14
Uruguay              9
Croatia              9
Slovenia             8
Moldova              7
Turkey               6
Bulgaria             5
Mexico               5
Cyprus               4
Lebanon              3
Georgia              3
Luxembourg           2
Macedonia            2
Brazil               2
England              1
Czech Republic       1
Slovakia             1
China                1
India                1
Serbia               1
Ukraine              1
Name: count, dtype: int64


In [9]:
min_country_count = 10  # Set the minimum count threshold
valid_countries = []

country_counts = data_first['country'].value_counts()
for country, count in country_counts.items():
    if count >= min_country_count:
        valid_countries.append(country)

# Filter the DataFrame to keep only the valid countries
data_first = data_first[data_first['country'].isin(valid_countries)]

country_counts = data_first['country'].value_counts()
print(country_counts)
print("Number of records:", len(data_first))


country
US              6135
Italy           1842
France          1444
Spain            873
Chile            599
Argentina        555
Australia        493
Portugal         437
New Zealand      298
Austria          234
Germany          221
South Africa     203
Greece            94
Israel            59
Hungary           26
Canada            25
Romania           14
Name: count, dtype: int64
Number of records: 13552


In [10]:
# Let's do the same for the columns 'province', 'variety', and 'winery'

# Providence column

min_province_count = 10  # Set the minimum count threshold
valid_provinces = []

province_counts = data_first['province'].value_counts()
for province, count in province_counts.items():
    if count >= min_province_count:
        valid_provinces.append(province)

data_first = data_first[data_first['province'].isin(valid_provinces)]

province_counts = data_first['province'].value_counts()
print(province_counts)
print("Number of records:", len(data_first))


province
California              4429
Washington               933
Tuscany                  584
Northern Spain           506
Mendoza Province         478
                        ... 
Judean Hills              11
Tokaji                    11
Península de Setúbal      10
Waipara                   10
Ribatejo                  10
Name: count, Length: 86, dtype: int64
Number of records: 13133


In [11]:
# For 'variety' column

min_variety_count = 10  # Set the minimum count threshold
valid_varieties = []

variety_counts = data_first['variety'].value_counts()
for variety, count in variety_counts.items():
    if count >= min_variety_count:
        valid_varieties.append(variety)

data_first = data_first[data_first['variety'].isin(valid_varieties)]

variety_counts = data_first['variety'].value_counts()
print(variety_counts)
print("Number of records:", len(data_first))

variety
Pinot Noir            1374
Chardonnay            1239
Cabernet Sauvignon    1214
Red Blend              903
Syrah                  565
                      ... 
Shiraz-Viognier         10
Austrian Red Blend      10
Muscat                  10
Pinot Nero              10
Fiano                   10
Name: count, Length: 89, dtype: int64
Number of records: 12552


In [12]:
# For 'winery' column - this column has far too much variation, so we will delete it
winery_counts = data_first['winery'].value_counts()
print(winery_counts)


data_first = data_first.drop('winery', axis=1)

print(data_first.columns)
print("Number of records:", len(data_first))

winery
Williams Selyem          34
Columbia Crest           29
Chateau Ste. Michelle    23
Trapiche                 23
Testarossa               23
                         ..
Tenuta Sette Ponti        1
La Fornace                1
Michael Pozzan            1
Michael Florentino        1
San Saba                  1
Name: count, Length: 5466, dtype: int64
Index(['country', 'points', 'price', 'province', 'variety'], dtype='object')
Number of records: 12552


Now, we need to calculate the price from the columns Country, Points, Province, and Variety

In [13]:
# Let's scale the points colunm
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the 'Points' column
data_first['points'] = scaler.fit_transform(data_first[['points']])
print(data_first.head(5))


     country  points  price          province             variety
0         US    0.15   10.0        California     Sauvignon Blanc
1         US    0.70   34.0        Washington               Syrah
2  Argentina    0.35   17.0  Mendoza Province  Cabernet Sauvignon
3      Spain    0.60   41.0    Northern Spain         Tempranillo
4      Chile    0.25   13.0  Aconcagua Valley  Cabernet Sauvignon


In [14]:
unique_varieties_count = data_first['variety'].nunique()
print("Number of unique values in the 'variety' column:", unique_varieties_count)

data_first = pd.get_dummies(data_first, columns=['variety', 'country', 'province'], prefix='variety')

print(len(data_first.columns))


Number of unique values in the 'variety' column: 89
191


In [15]:
# Now, let's build the model

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [16]:
X = data_first.drop('price', axis=1)
y = data_first['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

print('Accuracy:',rf_regressor.score(X_test, y_test)*100)


Accuracy: 30.111585726194413


Conclusion: Predicting the price of wines from the given data is unreasonable, as the price will be influenced by external data that we do not have on hand...